# 3. Orchestrating LLM Workflows with Chains

Chains are the heart of LangChain, allowing us to combine LLMs with other components to create sophisticated, multi-step applications. This notebook demonstrates how to build and execute various types of chains. We start with the fundamental `LLMChain`, move to linking chains together with `SequentialChain`, and finally implement conditional logic using a `RouterChain` to direct input to the most appropriate prompt based on the user's query.

In [83]:
import warnings
warnings.filterwarnings('ignore')

In [85]:
import os
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [86]:
llm_model = "gpt-4o"

In [87]:
df = pd.read_csv('Data.csv')

In [88]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [89]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [98]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Choose one name"
)

In [99]:
chain = LLMChain(llm=llm, prompt=prompt)

In [100]:
product = "Queen Size Sheet Set"
print(chain.run(product))

"QueenComfort Linens"


## SimpleSequentialChain

In [101]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Choose one name"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [102]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [103]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [104]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"QueenComfort Linens"
QueenComfort Linens offers luxurious, high-quality bedding and linens designed to enhance comfort and elevate your home’s aesthetic.

> Finished chain.


'QueenComfort Linens offers luxurious, high-quality bedding and linens designed to enhance comfort and elevate your home’s aesthetic.'

## SequentialChain

In [105]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [106]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [107]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [108]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [109]:
# overall_chain: input = Review 
# and output = English_Review, summary, followup_message 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "language", "summary", "followup_message"],
    verbose=True
)

In [110]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [111]:
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't last, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'language': 'The review is written in French.',
 'summary': 'The reviewer is disappointed with the mediocre taste and short-lasting foam of the product, suspecting it might be an old batch or counterfeit, especially since store-bought versions taste better.',
 'followup_message': "Cher/Chère [Nom du/ de la Relecteur/Relectrice],\n\nMerci d'avoir partagé votre expérience. Nous regrettons sincèrement que le goût et la qualité de notre produit n'aient pas répondu à vos attentes. Il est primordial pour nous de garantir la satisfaction de nos clients avec des produits de haute qualité. Pouvez-vous nous fournir plus de détails sur le lot que vo

## Router Chain

In [112]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [113]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [114]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [115]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [116]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [117]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [118]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [119]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True)

In [120]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the type of electromagnetic radiation emitted by an idealized object called a black body. A black body is an object that absorbs all incident radiation, regardless of frequency or angle of incidence, and emits radiation with a characteristic spectrum that depends only on its temperature.\n\nThe key features of black body radiation are:\n\n1. **Planck's Law**: The spectrum of black body radiation is described by Planck's Law, which gives the intensity of radiation at different wavelengths for a given temperature. The distribution shows that higher temperatures result in more radiation emitted and a shift of the peak intensity to shorter wavelengths.\n\n2. **Wien's Displacement Law**: This law states that the wavelength at which the emission of a black body peaks is inversely proportional to its temperature. So, as an object gets hotter, the peak of its emission spectrum moves to shorter wavelengths.\n\n3. **Stefan-Boltzmann Law**: This law quantifies the 

In [121]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve the problem of \\(2 + 2\\), we can break it down into a simple addition problem:\n\n1. **Identify the numbers involved**: The numbers we are adding are 2 and 2.\n\n2. **Perform the addition**: When you add 2 and 2 together, you combine them to find the total.\n\n\\[\n2 + 2 = 4\n\\]\n\nTherefore, the answer is 4.'

In [122]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because it serves as the fundamental genetic blueprint for building and maintaining the entire organism. DNA encodes the instructions necessary for the development, functioning, growth, and reproduction of all living cells and organisms. Here are several key reasons why nearly every cell contains DNA:\n\n1. **Genetic Information Storage**: DNA carries all the genetic instructions used in the growth and development of the organism. Each cell contains a complete set of DNA so it can perform its specific functions and contribute to the overall operation of the body.\n\n2. **Replication and Cell Division**: During cell division, DNA replication ensures that each new cell receives an exact copy of the genetic material. This continuity of genetic information is crucial for maintaining the organism's characteristics and functions.\n\n3. **Specialization and Differentiation**: While every cell contains the full DNA sequence, different genes are expressed in